# Covid-19 Model Training
Train the model with covid-19 datasets <br>
<br>
**Important note**: please run this code in Google Colab, but if you want to use other don't forget to adjust to their own version code

In [1]:
!pip install translate

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imageio
import pathlib
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
from tensorflow.keras import callbacks, optimizers
from google.colab import drive

%matplotlib inline

In [3]:
!ls

drive  sample_data


In [5]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [6]:
!ls

drive  sample_data


In [7]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [8]:
!ls

 2021-02-10-01-48-30.mp4		  'Day - 5.jpg'
'2401959023 - Andreas Alexander (1).zip'  'Day 5.jpg'
'2401959023 - Andreas Alexander.zip'	  'Day - 6.jpg'
'268407_PRESENTASI INGGRIS.gdoc'	  'Day 6.jpg'
 299872_script.gdoc			  'Day - 7.jpg'
 301049_script.gdoc			  'Day 7.jpg'
 Binus					  'Describe yourself.gdoc'
 CHAD.pptx				   IMG_0084.JPG
'Colab Notebooks'			   IMG_0090.JPG
 Covid_Datasets				   IMG_0091.JPG
'Day - 1.jpg'				   IMG_0101.JPG
'Day 1.jpg'				   IMG_0106.JPG
'Day - 2.jpg'				   IMG_0113.JPG
'Day 2.jpg'				   Pasfoto.jpg
'Day - 3.jpg'				   Pseudocode_Desk_Check.gdoc
'Day 3.jpg'				  'Untitled form (1).gform'
'Day - 4 .jpg'				  'Untitled form.gform'
'Day 4.jpg'


In [9]:
%cd Covid_Datasets

/content/drive/MyDrive/Covid_Datasets


In [10]:
from condition import condition

In [11]:
condition

{'negative': 'Negative', 'positive': 'Positive'}

In [13]:
os.listdir('train')

['NEGATIVE', 'POSITIVE']

In [14]:
!ls train

NEGATIVE  POSITIVE


In [15]:
for i in os.listdir('train/'):
  print(i, len(os.listdir('train/' + i)))

NEGATIVE 10266
POSITIVE 3690


In [16]:
def img_data(dir_path, target_size, batch, class_lst, preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()
  
  return (gen_object.flow_from_directory(dir_path, 
                                        target_size=target_size, 
                                        batch_size=batch, 
                                        class_mode='sparse', 
                                        classes=class_lst,
                                        shuffle = True))

In [17]:
train_data_gen = img_data("train/", (224, 224), 500, os.listdir('train'), preprocess_input)
valida_data_gen = img_data("test/", (224, 224), 500, os.listdir('test'), preprocess_input)

Found 13956 images belonging to 2 classes.
Found 13868 images belonging to 2 classes.


In [18]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000,
    classifier_activation='softmax'
)

In [19]:
base_model.trainable = False

In [20]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [21]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3,579,978
Trainable params: 1,321,994
Non-trainable params: 2,257,984
_________________________________________________________________


In [23]:
elst = callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_check = callbacks.ModelCheckpoint('.model.hdf5', save_best_only=True, monitor='val_loss', mode=min)


In [24]:
model.fit(train_data_gen, batch_size=500, validation_data=valida_data_gen, callbacks=[elst, save_check], epochs=10)

Epoch 1/10
28/28 [==============================] - ETA: 0s - loss: 0.7522 - accuracy: 0.7706 

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


28/28 [==============================] - 2688s 98s/step - loss: 0.7522 - accuracy: 0.7706 - val_loss: 0.2951 - val_accuracy: 0.8882
Epoch 2/10
28/28 [==============================] - 876s 32s/step - loss: 0.2251 - accuracy: 0.9117 - val_loss: 0.1789 - val_accuracy: 0.9319
Epoch 3/10
28/28 [==============================] - 861s 31s/step - loss: 0.1730 - accuracy: 0.9325 - val_loss: 0.1601 - val_accuracy: 0.9352
Epoch 4/10
28/28 [==============================] - 880s 32s/step - loss: 0.1515 - accuracy: 0.9426 - val_loss: 0.1314 - val_accuracy: 0.9501
Epoch 5/10
28/28 [==============================] - 875s 32s/step - loss: 0.1327 - accuracy: 0.9499 - val_loss: 0.1142 - val_accuracy: 0.9575
Epoch 6/10
28/28 [==============================] - 875s 32s/step - loss: 0.1149 - accuracy: 0.9580 - val_loss: 0.1017 - val_accuracy: 0.9649
Epoch 7/10
28/28 [==============================] - 847s 31s/step - loss: 0.1015 - accuracy: 0.9647 - val_loss: 0.0950 - val_accuracy: 0.9622
Epoch 8/10
28/28